<a href="https://colab.research.google.com/github/jburyk/Swiss_history_semantic_search/blob/main/Swiss_history_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install required packages


In [2]:
!pip install -qU langchain tiktoken matplotlib seaborn tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.9/709.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.3 MB/s eta 0:00:00


In [29]:
from getpass import getpass
OPEN_AI_API_KEY = getpass('Enter OpenAI key: ')
PINECONE_API_KEY = getpass('Enter Pinecone key: ')

Enter OpenAI key: ··········
Enter Pinecone key: ··········


In [22]:
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

inputText = open_file('Switzerland.txt')
inputText = inputText.replace('\n', ' ').replace('\r', '')


In [23]:
import tiktoken

def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

tokenizer = tiktoken.get_encoding('cl100k_base')

In [24]:
tiktoken_len(inputText)

16674

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=150,
        chunk_overlap=10,
        length_function=tiktoken_len,
        separators=["\n\n", "\n", " ", ""]
    )

chunks = text_splitter.split_text(inputText)

In [27]:
chunks[0]

'Switzerland’s history is one of a medieval defensive league formed during a time and in an area lacking imperial authority. The different cantons (traditionally called Orte in German) were to a large extent independent states that remained united through the shared defense of liberty, which was understood as the protection of imperial privileges and franchises. Unlike all similar confederations (e.g., the'

In [20]:
from getpass import getpass
secret = getpass('Enter the secret value: ')

Enter the secret value: ··········


In [21]:
secret


'ala ma kota'